# Deep Learning Computation

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

## Layers and blocks

In [2]:
net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

X = torch.rand(2, 20)
net(X)

tensor([[-0.0042, -0.0571, -0.0960,  0.0077,  0.1813, -0.2693,  0.0019, -0.1460,
         -0.0239,  0.0881],
        [-0.0003, -0.0493, -0.0187,  0.1120,  0.1079, -0.2407, -0.0052, -0.1455,
         -0.0342,  0.0969]], grad_fn=<AddmmBackward>)

In [3]:
# a custom block
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
    
net = MLP()
net(X)

tensor([[ 0.2010,  0.0099, -0.1108,  0.1824,  0.1185,  0.0998,  0.2935, -0.0237,
         -0.0726, -0.0374],
        [ 0.2339, -0.0212, -0.0951,  0.1982,  0.1339,  0.0976,  0.1462, -0.0567,
         -0.0223, -0.0265]], grad_fn=<AddmmBackward>)

In [4]:
# the sequential block
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
            
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0279,  0.0390,  0.1892, -0.2149,  0.0491,  0.0151, -0.1824, -0.0728,
          0.0254, -0.0632],
        [ 0.0005,  0.1743,  0.0114, -0.2017,  0.0195,  0.1298, -0.1718, -0.1265,
         -0.0259,  0.0011]], grad_fn=<AddmmBackward>)

In [5]:
# executing code in the forward propagation function
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # constant weight
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(0.1761, grad_fn=<SumBackward0>)

In [6]:
class NestedMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))
    
net = nn.Sequential(NestedMLP(), nn.Linear(16, 20), FixedHiddenMLP())
net(X)

tensor(0.4001, grad_fn=<SumBackward0>)

## Parameter management

In [7]:
# parameter access
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand((2, 4))
net[2].state_dict()

OrderedDict([('weight',
              tensor([[-0.2512, -0.2893,  0.1572,  0.3387, -0.3104,  0.2261,  0.1779, -0.0692]])),
             ('bias', tensor([-0.0140]))])

In [8]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0140], requires_grad=True)
tensor([-0.0140])


In [9]:
net[2].weight.grad == None

True

In [10]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [11]:
net.state_dict()['2.bias'].data

tensor([-0.0140])

In [12]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [13]:
# parameter initialization
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
        
net.apply(init_normal)
net[0].weight.mean(), net[0].weight.std()

(tensor(-0.0007, grad_fn=<MeanBackward0>),
 tensor(0.0098, grad_fn=<StdBackward>))

In [14]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [15]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
    
net[0].apply(xavier)
print(net[0].weight.data)

tensor([[-0.6716,  0.4426, -0.0030, -0.2245],
        [-0.1212, -0.5738, -0.0959,  0.5338],
        [ 0.6569,  0.4626, -0.5875,  0.2235],
        [ 0.1284, -0.3802,  0.5871,  0.6683],
        [ 0.1283,  0.2448, -0.6528, -0.3597],
        [-0.3060,  0.5954,  0.6770,  0.3039],
        [-0.3235,  0.2223,  0.5281, -0.6617],
        [ 0.5173,  0.4232, -0.1105,  0.6679]])


In [16]:
# shared parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(
    nn.Linear(4, 8), 
    nn.ReLU(),
    shared, 
    nn.ReLU(),
    shared,
    nn.ReLU(),
    nn.Linear(8, 1)
)
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


## Custom layers

In [17]:
# layers without parameters
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()
    
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [18]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-3.7253e-09, grad_fn=<MeanBackward0>)

In [19]:
# layers with parameters
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(in_units, units))
        self.bias = nn.Parameter(torch.rand(units,))
        
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[0.6007, 0.8526, 0.3262],
        [0.3498, 0.6332, 0.6822],
        [0.3088, 0.1676, 0.8219],
        [0.2370, 0.0142, 0.5184],
        [0.8655, 0.4789, 0.6026]], requires_grad=True)

In [20]:
linear(torch.rand(2, 5))

tensor([[2.1517, 1.5823, 1.9894],
        [1.3476, 0.6026, 1.0518]])

In [21]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[56.4579],
        [54.0867]])

## File I/O

In [22]:
# loading and saving tensors
x = torch.arange(4)
torch.save(x, 'x-file')
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [23]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
x2, y2

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [24]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [25]:
# loading and saving model parameters
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
        
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
torch.save(net.state_dict(), 'mlp.params')

X = torch.randn(size=(2, 20))
Y = net(X)

clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])